Steps of ABC:

1- Generate initial randomized population an evaluate them.

2- Movement of recruited bees.

3- Send onlooker bees. (Calculate probabilities -> Select site -> Generate new solution -> Evaluation)

4- If there is a site which is reached to the the number of unproceeded counts, that site will be replaced randomly with another one.

5- If end conditions are not met yet, return to the step 2, else the end.

This algorithm just works for continuous space.


In [35]:
#import libraries
import numpy as np
import copy
import random as rd
import matplotlib.pyplot as plt

In [36]:
#define function

def sphere(x):
    return np.sum(np.power(x,2))

In [37]:
#problem definition
def costFunction(x):
    return sphere(x)

nVar = 5    #number of decision variable
varSize = nVar #decision variable matrix size
varMin = -10    # decision variable lower bound,  can be changed to array
varMax = 10     #decision variable upper bound, can be changed to array


In [38]:
#ABC Settings
nPop  = 10      # number of population, scout B

nOnlooker = nPop    #number of onlookers bees

maxIt = 100     #Maximum number of iteration

a_limit = int(round(0.5 *nPop * nVar,0))     #Abandonment Limit Parameter

acu = 1       #acceleration coefficient upper bound


In [86]:
#Initialization

#empty_bee structure
bee_positions = []
bee_costs = []

#Initialize best solution ever found
bestSol_cost = np.inf
bestSol_position = []

#create initial population
for i in range(nPop):
    position = np.random.uniform(varMin,varMax,nVar).tolist()
    bee_positions.append(position)

    cost = costFunction(position)
    bee_costs.append(cost)

    if cost <= bestSol_cost:
        bestSol_cost = cost
        bestSol_position = position

#Abandonment Counter
C = np.zeros((nPop,1))

#List of best costs in iteration
BestCost = []

In [40]:
def roulettWheelSelection(P):
    rNum = rd.random()
    C = np.cumsum(P)
    return min(np.where(rNum <= C)[0])

In [85]:
#ABC Main loop
for it in range(maxIt):
    #Recruited Bees
    for i in range(nPop):
        newPopPosition = copy.deepcopy(bee_positions)
        newPopCost =  copy.deepcopy(bee_costs)

        K = [x for x in range(nPop) if x !=i]
        #random number from K not equal to i
        k =  K[rd.randint(0,len(K)-1)]

        #define accelaration coef
        phi = np.random.uniform(-acu, acu, varSize)

        #new bee position
        multi_List = [a - b for a, b in zip(bee_positions[i], bee_positions[k])]
        newPopPosition = bee_positions[p] + [s * u for s,u in zip(phi,multi_List)]

        #Evaluation
        newPopCost = costFunction(position)

        #Comparision
        if newPopCost <= bee_costs[i]:
            bee_positions[i] = newPopPosition
            bee_costs[i] = newPopCost
        else:
            C[i] += C[i]


    #calculate fitness values and selection probabilities
    FF = np.zeros((nPop,1))
    for i in range(nPop):
        if bee_costs[i] >=0:
            FF[i] = 1 / (1 + bee_costs[i])
        else:
            FF[i] = 1 + abs((1 + bee_costs[i]))

    P = FF / sum(FF)

    #onlooker Bees
    for ol in range(nOnlooker):
        #select source site
        p = roulettWheelSelection(P)

        K = [x for x in range(nPop) if x !=p]
        #random number from K not equal to i
        k =  K[rd.randint(0,len(K)-1)]

        #define accelaration coef
        phi = np.random.uniform(-acu, acu, varSize).tolist()

        #new bee position
        multi_List = [a - b for a, b in zip(bee_positions[i], bee_positions[k])]
        newPopPosition = bee_positions[p] + [s * u for s,u in zip(phi,multi_List)]
        #Evaluation
        newPopCost = costFunction(newPopPosition)

        #Comparision
        if newPopCost <= bee_costs[p]:
            bee_positions[p] = newPopPosition
            bee_costs[p] = newPopCost
        else:
            C[p] += C[p]

    #scout Bees
    for i in range(nPop):
        if C[i] >= a_limit:
            position = np.random.uniform(varMin,varMax,nVar).tolist()
            bee_positions.append(position)

            cost = costFunction(position)
            bee_costs.append(cost)

            C[i] = 0

    #Update best solution ever found
    for i in range(nPop):
        if bee_costs[i] <= bestSol_cost:
            bestSol_position = bee_positions
            bestSol_cost = bee_costs[i]

    #Store best cost ever cost
    BestCost.append(bestSol_cost)

    #Show Iteration Information
    print('Iteration ' + str(it) +": Best Cost = " + str(BestCost[it]))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
#Plot the cost results
plt.figure(figsize=(15, 10))
plt.plot(BestCost, 'ro-', label='Cost Results', linewidth=3, markersize=3)
plt.title('Results for BBO Algorithm')
plt.ylabel('Best Cost')
plt.xlabel('Iteration')
plt.show()

In [84]:
bestSol_cost

[134.5102557679421,
 187.17632863505264,
 92.14354748300278,
 266.4050886932922,
 139.55248669134335,
 264.9870202204546,
 157.1195521909191,
 176.77365270245417,
 224.87414569882802,
 267.6410642479568]

In [66]:
* ([a - b for a, b in zip(bee_positions[i], bee_positions[k])])

array([  1.59214123,  -4.8786129 , -10.15019461,  -2.07323559,
        -1.8517159 ])

In [75]:
phi 

[0.3395445874844092,
 -0.4930073226597489,
 0.7861164448070574,
 0.5813313387978534,
 0.6490006831970321]